In [194]:
# Import Meteostat library and dependencies
from datetime import datetime
import matplotlib.pyplot as plt
from meteostat import Point, Daily
import pandas as pd
pd.set_option('display.max_columns', None)

# Set time period
start = datetime(2010, 1, 1)
end = datetime(2023, 12, 31)

lax = Point(33.9416, -118.4085)
lax_data = Daily(lax, start, end)
lax_data = lax_data.fetch()

sfo = Point( 37.619, -122.375)
sfo_data = Daily(sfo, start, end)
sfo_data = sfo_data.fetch()

for column in sfo_data.columns:
    sfo_data = sfo_data.rename(columns={column : 'sfo_' + column})
    lax_data = lax_data.rename(columns={column : 'lax_' + column})
    
data = sfo_data.merge(lax_data, on='time')

# no data in these columns at all
data = data.drop(['lax_wpgt', 'sfo_wpgt', 'lax_tsun', 'sfo_tsun', 'lax_snow', 'sfo_snow'], axis=1)

# whole months of missing wind speed directions + direction seems random
data = data.drop(['lax_wdir', 'sfo_wdir'], axis=1)

# Plot line chart including average, minimum and maximum temperature
# lax_data.plot(y=['tavg', 'tmin', 'tmax'])
# plt.show()

In [195]:
# only a few missing

data['lax_pres'].interpolate(inplace=True)
data['sfo_pres'].interpolate(inplace=True)
data['lax_tavg'].interpolate(inplace=True)
data['sfo_tavg'].interpolate(inplace=True)

# 3 missing
data['lax_prcp'].fillna(0, inplace=True)
data['sfo_prcp'].fillna(0, inplace=True)

In [196]:
df = pd.read_csv('../data/airlines_filtered_selected_with_delayrates.csv')

df['RoundFlightDate'] = pd.to_datetime(df.FlightDate).dt.round('d')
df.set_index('RoundFlightDate', inplace=True)

merged = df.merge(data, left_index=True, right_index=True, how='left')
merged.to_csv('../data/airlines_filtered_selected_with_weather_delayrates.csv', index=False, encoding='utf-8')
merged = pd.read_csv('../data/airlines_filtered_selected_with_weather_delayrates.csv')

In [197]:
# nan rate
merged.isna().sum() / merged.shape[0]

Year                 0.000000
Month                0.000000
DayofMonth           0.000000
DayOfWeek            0.000000
FlightDate           0.000000
Reporting_Airline    0.000000
Tail_Number          0.000000
Origin               0.000000
Dest                 0.000000
DepTime              0.000000
DepDelay             0.000000
DepDelayMinutes      0.000000
DepDel15             0.000000
DepTimeBlk           0.000000
ArrTime              0.000000
ArrDelay             0.000293
ArrDelayMinutes      0.000293
ArrDel15             0.000293
ArrTimeBlk           0.000007
Cancelled            0.000000
CancellationCode     1.000000
AirTime              0.000293
CarrierDelay         0.744885
WeatherDelay         0.744885
NASDelay             0.744885
SecurityDelay        0.744885
LateAircraftDelay    0.744885
SFODepDelRate        0.000000
LAXDepDelRate        0.000000
SFOArrDelRate        0.000000
LAXArrDelRate        0.000000
sfo_tavg             0.000000
sfo_tmin             0.000000
sfo_tmax  

# Handles NANs

In [198]:
merged.loc[:, ['ArrDelay', 'ArrDel15', 'ArrDelayMinutes']].dropna(inplace=True)
merged.isna().sum() / merged.shape[0]

Year                 0.000000
Month                0.000000
DayofMonth           0.000000
DayOfWeek            0.000000
FlightDate           0.000000
Reporting_Airline    0.000000
Tail_Number          0.000000
Origin               0.000000
Dest                 0.000000
DepTime              0.000000
DepDelay             0.000000
DepDelayMinutes      0.000000
DepDel15             0.000000
DepTimeBlk           0.000000
ArrTime              0.000000
ArrDelay             0.000293
ArrDelayMinutes      0.000293
ArrDel15             0.000293
ArrTimeBlk           0.000007
Cancelled            0.000000
CancellationCode     1.000000
AirTime              0.000293
CarrierDelay         0.744885
WeatherDelay         0.744885
NASDelay             0.744885
SecurityDelay        0.744885
LateAircraftDelay    0.744885
SFODepDelRate        0.000000
LAXDepDelRate        0.000000
SFOArrDelRate        0.000000
LAXArrDelRate        0.000000
sfo_tavg             0.000000
sfo_tmin             0.000000
sfo_tmax  

In [199]:
# Can't have label missing
merged.dropna(subset=['ArrDel15'], inplace=True)

irrelevant = ['CancellationCode', 
              'AirTime', 
              'CarrierDelay', 
              'WeatherDelay', 
              'NASDelay', 
              'SecurityDelay', 
              'LateAircraftDelay',
              'ArrDelay',
              'ArrDelayMinutes',
              'ArrTimeBlk',
              'Origin',
              'Dest',
              'DepDelay', 
              'DepDelayMinutes', 
              'DepDel15', 
              'DepTimeBlk',
              'Cancelled',
              'FlightDate'
              ]

# drop irrelevant features
merged.drop(
    columns=irrelevant,
    inplace=True,
    axis=1,
)

In [200]:
print(merged.columns)

Index(['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'Reporting_Airline',
       'Tail_Number', 'DepTime', 'ArrTime', 'ArrDel15', 'SFODepDelRate',
       'LAXDepDelRate', 'SFOArrDelRate', 'LAXArrDelRate', 'sfo_tavg',
       'sfo_tmin', 'sfo_tmax', 'sfo_prcp', 'sfo_wspd', 'sfo_pres', 'lax_tavg',
       'lax_tmin', 'lax_tmax', 'lax_prcp', 'lax_wspd', 'lax_pres'],
      dtype='object')


## One hot encode airline and tail_number previous delay %

In [201]:
# this will be stacked with the standardized feature matrix
airline_enc = pd.get_dummies(merged['Reporting_Airline'])
# tail_number is one hot encoded based on previous history of % of flights that particular airplane has been delayed.

merged['delayed_cumulative_sum'] = merged.groupby('Tail_Number')['ArrDel15'].cumsum()
merged['delayed_cumulative_count'] = merged.groupby('Tail_Number')['ArrDel15'].cumcount() + 1
merged['prev_tail_delay_percentage'] = merged['delayed_cumulative_sum'] / merged['delayed_cumulative_count']
# merged['delay_percentage_20'] = np.where(merged['delay_percentage'] >= .2, 1, 0)
# tailnum_enc = pd.get_dummies(merged['delay_percentage_20'])
# tailnum_enc.columns = ['tail_number_good_delay_history','tail_number_bad_delay_history']
merged.drop(columns=['Reporting_Airline', 'Tail_Number','delayed_cumulative_sum','delayed_cumulative_count'], axis=1, inplace=True)

In [202]:
merged

,Year,Month,DayofMonth,DayOfWeek,DepTime,ArrTime,ArrDel15,SFODepDelRate,LAXDepDelRate,SFOArrDelRate,LAXArrDelRate,sfo_tavg,sfo_tmin,sfo_tmax,sfo_prcp,sfo_wspd,sfo_pres,lax_tavg,lax_tmin,lax_tmax,lax_prcp,lax_wspd,lax_pres,prev_tail_delay_percentage
0,2010,1,1,5,556.0,725.0,0.0,0.000000,0.000000,0.000000,0.000000,11.8,10.0,13.9,0.0,4.7,1026.1,15.2,10.0,20.6,0.0,7.6,1022.1,0.000000
1,2010,1,1,5,755.0,903.0,0.0,0.551083,0.000000,0.000000,0.000000,11.8,10.0,13.9,0.0,4.7,1026.1,15.2,10.0,20.6,0.0,7.6,1022.1,0.000000
2,2010,1,1,5,756.0,918.0,0.0,0.551083,0.000000,0.000000,0.000000,11.8,10.0,13.9,0.0,4.7,1026.1,15.2,10.0,20.6,0.0,7.6,1022.1,0.000000
3,2010,1,1,5,758.0,925.0,0.0,0.551083,0.000000,0.000000,0.000000,11.8,10.0,13.9,0.0,4.7,1026.1,15.2,10.0,20.6,0.0,7.6,1022.1,0.000000
4,2010,1,1,5,918.0,1047.0,0.0,0.000000,0.000000,0.000000,0.000000,11.8,10.0,13.9,0.0,4.7,1026.1,15.2,10.0,20.6,0.0,7.6,1022.1,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150104,2020,3,31,2,1910.0,2019.0,0.0,0.016972,0.100879,0.145184,0.030149,12.7,9.4,15.6,0.0,25.9,1016.2,17.3,13.9,20.6,0.0,10.8,1011.0,0.461538
150105,2020,3,31,2,2025.0,2135.0,0.0,0.129517,0.095327,0.061612,0.017893,12.7,9.4,15.6,0.0,25.9,1016.2,17.3,13.9,20.6,0.0,10.8,1011.0,0.000000
150106,2020,3,31,2,2107.0,2216.0,0.0,0.125039,0.111775,0.027268,0.070302,12.7,9.4,15.6,0.0,25.9,1016.2,17.3,13.9,20.6,0.0,10.8,1011.0,0.303571
150107,2020,3,31,2,2117.0,2236.0,0.0,0.125039,0.111775,0.027268,0.070302,12.7,9.4,15.6,0.0,25.9,1016.2,17.3,13.9,20.6,0.0,10.8,1011.0,0.142857


## Standardize data

In [203]:
from sklearn.preprocessing import StandardScaler
import numpy as np


X = merged.drop(columns='ArrDel15', axis=1).to_numpy()
y = merged['ArrDel15'].to_numpy()

scaler = StandardScaler()
X = scaler.fit_transform(X)

# put hot encoded airline and label back in
X = np.hstack((X, airline_enc.to_numpy(), y.reshape(-1, 1)))
np.savetxt('../data/airline_final.csv', X, delimiter=',')
with open('../data/airline_final.npy', 'wb') as f:
    np.save(f, X)